In [297]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [298]:
transfer=pd.read_parquet('transfers.parquet')
transfer

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,None,2180-05-07 17:21:27,None
3,10000032,22841357.0,34100253,discharge,None,2180-06-27 18:49:12,None
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
...,...,...,...,...,...,...,...
1890967,19999914,NaN,32002659,ED,Emergency Department,2158-12-24 11:41:00,2158-12-24 11:56:00
1890968,19999987,23865745.0,30249304,transfer,Neurology,2145-11-04 21:29:30,2145-11-11 13:00:47
1890969,19999987,23865745.0,34731548,ED,Emergency Department,2145-11-02 19:28:00,2145-11-02 22:59:00
1890970,19999987,23865745.0,36195440,admit,Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30


In [299]:
transfer = transfer.sort_values(by=['hadm_id', 'intime']).reset_index(drop=True)

In [300]:
icu_list=['Trauma SICU (TSICU)', 'Neuro Surgical Intensive Care Unit (Neuro SICU)','Coronary Care Unit (CCU)', 'Cardiac Vascular Intensive Care Unit (CVICU)', 'Medical/Surgical Intensive Care Unit (MICU/SICU)', 'Surgical Intensive Care Unit (SICU)', 'Medical Intensive Care Unit (MICU)']

# MED hadm_id

In [309]:
import pandas as pd
from tqdm import tqdm

nor_hadm_ids = []

for hadm_id in tqdm(transfer['hadm_id'].unique()):
    # Filter rows for the current hadm_id
    tmp = transfer[transfer['hadm_id'] == hadm_id]
    
    if len(tmp) > 1:
        # Check the first and last row's conditions and middle rows for 'Medicine'
        if (tmp.iloc[0]['eventtype'] == 'admit' and tmp.iloc[0]['careunit'] == 'Medicine' and
            tmp.iloc[-1]['eventtype'] == 'discharge' and (tmp.iloc[1:-1]['careunit'] == 'Medicine').all()):
            nor_hadm_ids.append(hadm_id)
            #print('1')
        if len(tmp) > 2:
            if (tmp.iloc[1]['eventtype'] == 'admit' and tmp.iloc[1]['careunit'] == 'Medicine' and
                tmp.iloc[-1]['eventtype'] == 'discharge' and (tmp.iloc[2:-1]['careunit'] == 'Medicine').all()):
                nor_hadm_ids.append(hadm_id)
                #print('2')
print("Valid HADM IDs meeting the criteria:", nor_hadm_ids)


100%|██████████| 431232/431232 [10:51<00:00, 662.19it/s]

Valid HADM IDs meeting the criteria: [20000019.0, 20000024.0, 20000034.0, 20000057.0, 20000254.0, 20000343.0, 20000750.0, 20001002.0, 20001122.0, 20001353.0, 20001633.0, 20001720.0, 20001789.0, 20001984.0, 20002027.0, 20002054.0, 20002287.0, 20002634.0, 20002800.0, 20003013.0, 20003070.0, 20003239.0, 20003563.0, 20004038.0, 20004487.0, 20004686.0, 20004785.0, 20004829.0, 20005361.0, 20005497.0, 20005525.0, 20005666.0, 20005914.0, 20005915.0, 20006303.0, 20006517.0, 20006820.0, 20006862.0, 20006989.0, 20007246.0, 20007700.0, 20007709.0, 20008232.0, 20008236.0, 20008372.0, 20008598.0, 20008865.0, 20009020.0, 20009173.0, 20009260.0, 20009487.0, 20009652.0, 20009782.0, 20009795.0, 20010021.0, 20010097.0, 20010350.0, 20010647.0, 20010709.0, 20010748.0, 20011234.0, 20011498.0, 20011503.0, 20011674.0, 20012135.0, 20012747.0, 20012880.0, 20013280.0, 20013317.0, 20013426.0, 20013465.0, 20013681.0, 20014035.0, 20014311.0, 20014344.0, 20014583.0, 20014672.0, 20014856.0, 20014929.0, 20014934.0, 20

In [310]:
len(nor_hadm_ids)

64531

# MICU hadm_id

In [303]:
import pandas as pd
from tqdm import tqdm

ab_hadm_ids = []

for hadm_id in tqdm(transfer['hadm_id'].unique()):
    tmp = transfer[transfer['hadm_id'] == hadm_id]
    
    if len(tmp) > 1:
        # Check the first and last row's conditions
        if (tmp.iloc[0]['eventtype'] == 'admit' and tmp.iloc[0]['careunit'] == 'Medicine' and tmp.iloc[-1]['eventtype'] == 'discharge'):
            middle_care_units = list(tmp.iloc[1:-1]['careunit'].values)
            # Check if MICU is present in the middle care units
            if 'Medical Intensive Care Unit (MICU)' in middle_care_units:
                first_micu_index = middle_care_units.index('Medical Intensive Care Unit (MICU)')
                # Check all preceding entries are 'Medicine'
                if all(unit == 'Medicine' for unit in middle_care_units[:first_micu_index]):
                    #print(str(first_micu_index)+'_1')
                    ab_hadm_ids.append(hadm_id)
        
        # Additional check if there are at least three rows to check second row separately
        if len(tmp) > 2:
            if tmp.iloc[1]['eventtype'] == 'admit' and tmp.iloc[1]['careunit'] == 'Medicine':
                middle_care_units = list(tmp.iloc[2:-1]['careunit'].values)
                if 'Medical Intensive Care Unit (MICU)' in middle_care_units:
                    first_micu_index = middle_care_units.index('Medical Intensive Care Unit (MICU)')
                    if all(unit == 'Medicine' for unit in middle_care_units[:first_micu_index]):
                        #print(str(first_micu_index)+'_2')
                        ab_hadm_ids.append(hadm_id)

print("Valid HADM IDs meeting the criteria:", ab_hadm_ids)


  0%|          | 0/431232 [00:00<?, ?it/s]

100%|██████████| 431232/431232 [10:21<00:00, 694.35it/s]

Valid HADM IDs meeting the criteria: [20023461.0, 20026981.0, 20034762.0, 20098037.0, 20108677.0, 20120859.0, 20120958.0, 20130574.0, 20146445.0, 20147174.0, 20147685.0, 20163287.0, 20184300.0, 20194161.0, 20196119.0, 20204877.0, 20206823.0, 20208866.0, 20217604.0, 20224456.0, 20232963.0, 20246769.0, 20306191.0, 20307579.0, 20328020.0, 20329411.0, 20342223.0, 20389542.0, 20390569.0, 20405255.0, 20413690.0, 20435856.0, 20437208.0, 20459901.0, 20493144.0, 20494256.0, 20503127.0, 20522049.0, 20530843.0, 20531517.0, 20532512.0, 20541718.0, 20548929.0, 20561341.0, 20572421.0, 20572505.0, 20584551.0, 20592323.0, 20604937.0, 20619571.0, 20649773.0, 20650489.0, 20657068.0, 20657195.0, 20672350.0, 20676943.0, 20721409.0, 20724405.0, 20733692.0, 20738984.0, 20764485.0, 20784786.0, 20794985.0, 20797308.0, 20797390.0, 20815478.0, 20819317.0, 20830456.0, 20831510.0, 20844968.0, 20847700.0, 20849590.0, 20849804.0, 20856470.0, 20865228.0, 20879199.0, 20887418.0, 20892451.0, 20892999.0, 20895103.0, 20

In [304]:
len(ab_hadm_ids)

1030

In [308]:
nor_hadm_ids

[]

In [305]:
set(ab_hadm_ids)&set(nor_hadm_ids)

set()

In [318]:
import pickle

with open('MED_MICU.pkl', 'wb') as f:
    pickle.dump({'ab_hadm_ids': ab_hadm_ids, 'nor_hadm_ids': nor_hadm_ids}, f)